In [ ]:
import mxnet as mx
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from Symbol.symbol import get_resnet_model_YoloV1
from Symbol.symbol import Yolo_loss

import cv2

In [ ]:
def decodeBox(yolobox, size, dscale):
    i, j, cx, cy, w, h, cls1, cls2, cls3, cls4 = yolobox
    cxt = j*dscale + cx*dscale
    cyt = i*dscale + cy*dscale
    wt = w*size
    ht = h*size
    # true location in image
    cls_index = np.argmax([cls1, cls2, cls3, cls4])

    if cls_index==0:
        cls=1
    elif cls_index==1:
        cls=2
    elif cls_index==2:
        cls=3
    elif cls_index==3:
        cls=20
    return [cxt, cyt, wt, ht, cls]

def bboxdraw(img, label, dscale=32):
    assert label.shape == (7,7,9)
    size = img.shape[1]
    ilist, jlist = np.where(label[:,:,0]>0.5)
  
    
    # Create figure and axes
    fig,ax = plt.subplots(1)
    ax.imshow(np.uint8(img))
    bbox_list=[]
    for i,j in zip(ilist, jlist): 
        cx,cy,w,h,cls1, cls2, cls3, cls4 = label[i,j,1:]
        cxt, cyt, wt ,ht, cls = decodeBox([i, j, cx,cy,w,h,cls1, cls2, cls3, cls4], size, dscale)
        prob = label[i,j,0]
        bbox_list.append([cxt, cyt, wt ,ht, cls,prob])
        
        # Create a Rectangle patch
        rect = patches.Rectangle((cxt-wt/2,cyt-ht/2), wt,ht,linewidth=1,edgecolor='r',facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)
    
        name="unkown"
        if cls==1:
            name="{}".format(int(prob*100)/100)+"car"
        elif cls==2:
            name="{}".format(int(prob*100)/100)+"pedestrian"
        elif cls==3:
            name="{}".format(int(prob*100)/100)+"cyclist"
        elif cls==20:
            name="{}".format(int(prob*100)/100)+"traffic lights"
        print(name)


        plt.text(x=int(cxt-wt/2), y=int(cyt-ht/2), s=str(name), bbox=dict(facecolor='red', alpha=0.5))
    plt.show()
    return np.float64(bbox_list).tolist()

In [ ]:
imglist = []
for i in range(2000):
    imglist.append("{}".format(70091+i)+".jpg")
print(len(imglist))
print(imglist[:5])

In [ ]:
# get sym
sym, args_params, aux_params = mx.model.load_checkpoint("models/drive_detect", 9365)
logit = sym.get_internals()['logit_output']   #(-1,7,7,9)
#logit.infer_shape(data=(13,3,224,224))
#mx.viz.plot_network(logit,shape={"data":(13,3,224,256)})
mod = mx.mod.Module(symbol=logit,context =mx.cpu())
data_shape=[('data', (1,3,224,224))]
mod.bind(data_shapes=data_shape)
mod.init_params(allow_missing=False, arg_params=args_params, aux_params=aux_params)

    #predict test

In [ ]:
test_data_path = "testDATA/testing/"
test_result={}
for i in imglist:
    print(i)
    img_test = plt.imread(test_data_path+i)
#     plt.imshow(img_test)
#     plt.show()
#     print("orignal Img")
#     print(img_test.shape)
    W = 224
    H = 224
    img_test_resize = cv2.resize(img_test,(W,H))
#     plt.imshow(img_test_resize)
#     print("Resize Img:",i)
#     plt.show()
    
    # prediction:
    img_test_nd = mx.nd.array(ctx=mx.cpu(),source_array =img_test_resize.transpose((2,0,1)).reshape(1,3,H,W))
    img_itr = mx.io.NDArrayIter(data =img_test_nd,data_name="data",batch_size=1)
    # bind data to symbol
#     mod.bind(img_itr.provide_data)
#     mod.init_params(allow_missing=False, arg_params=args_params, aux_params=aux_params)
    #predict test
    # out = mod.predict(eval_data=train_data, num_batch=3)
    out = mod.predict(eval_data=img_itr)
    #pred->softsing->x/(1+abs(x))
    pred = (out.asnumpy()[0]+1)/2
#     print(pred.shape)
#     print(pred)
    bbox_list=bboxdraw(img_test_resize,pred)
#     print(bbox_list)
    test_result[i] =bbox_list

    

In [ ]:
import json
json = json.dumps(test_result)
f = open("test_result.json","w")
f.write(json)
f.close()

# import json
 
# dict = {'Python' : '.py', 'C++' : '.cpp', 'Java' : '.java'}
 
# json = json.dumps(dict)
# f = open("dict.json","w")
# f.write(json)
# f.close()

# print(test_result)